In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.stats import mode,zscore
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

data = pd.read_json("../../../../data/data.json")

data= data[data["capture"]== "tw2"]

data.sample(2)

,index,mac,timestamp,rssi,frequency,iq,angle,distance,capture,switch_order,ref,phases,aoa
23305,290,c300000a7a48,2024-11-29 16:39:44.416000+00:00,-47,2402,"[-144, -59, 90, -124, 103, 119, -128, 92, -78,...",-40,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[-2.7527332488000003, -0.942959733, 0.85734583...","[0.3114416413, 2.1216012103, -2.331809081, -0....","[-36.0252343857, -35.5301209109, -27.914896283..."
40041,87,c300000a7a48,2024-11-29 18:37:11.877000+00:00,-63,2480,"[-165, 9, 8, -165, 161, 37, -59, 156, -141, -8...",75,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[3.0871011973, -1.5223494172, 0.2258914073, 1....","[-0.22631830990000001, -0.1828655389, -1.10005...","[90.0, 43.0572263145, 30.5555589462, 66.748372..."


# Coluna

Esse modelo usa como entrada cada coluna IQ que sabemos que existe no pacote, logo precisamos gerá-las

In [2]:
#2-2-2-0-5-6-14-13-8-10 = 8*2 elementos em cada coluna, 2*8 de ref
# CTE = 20*8 => 160-4-8= 148 => 148*2 = 74 =>74//8 => 9 com resto 2, logo precisamos ignorar a última amostra 
data["training"] = data["phases"].map(lambda x: [np.array(x[i:i+4])[1:]-np.array(x[i:i+4])[:-1] for i in range(0,len(x),4)][:-1]) 
len(data["training"].iloc[0])

18

In [3]:
tsamples =[]
tlabels =[]

vsamples=[]
vlabels=[]

# Separamos no dataset para que a gnt seja capaz de depois analizar o conteúdo por pacote
tdataset, vdataset = train_test_split(data,test_size=0.3,random_state=10)

for row in tdataset.iterrows():
    for element in row[1]["training"]:
        tsamples.append(element)
        tlabels.append(row[1]["angle"])

for row in vdataset.iterrows():
    for element in row[1]["training"]:
        vsamples.append(element)
        vlabels.append(row[1]["angle"])

X_train = tsamples
X_val = vsamples


y_train = tlabels
y_val = vlabels 


In [4]:
tv=pd.DataFrame([{"angulo":angle, "treinamento":y_train.count(angle),"validacao" :y_val.count(angle)} for angle in set(y_val)])
tv.set_index("angulo")

,treinamento,validacao
angulo,,
0,18954,8100
1,3618,1800
2,3780,1638
3,3978,1440
4,3960,1458
5,3726,1692
6,3672,1746
7,3654,1764
8,3942,1476


In [5]:
model = xgb.XGBRegressor(enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [6]:
#Making predictions on the test set
predictions = model.predict(X_val)

#Calculating accuracy
mae_model =mean_absolute_error(y_val, predictions)
mse_model =mean_squared_error(y_val, predictions)

print(f"MAE= Modelo: {mae_model}")
print(f"MSE= Modelo: {mse_model}")

MAE= Modelo: 5.809972406957536
MSE= Modelo: 130.49173314234935


In [7]:
aoa_val=[]
results_labels= []
results_values = []
for row in vdataset.iterrows():
    pred=[]
    for element in row[1]["training"]:
        pred.append(model.predict([element])[0])
    results_labels.append((row[1]["angle"]))
    results_values.append(np.mean(pred))
    aoa_val.append(np.mean(row[1]["aoa"]))

In [8]:
#Calculating accuracy

mae_model =mean_absolute_error(results_labels, results_values)
mse_model =mean_squared_error(results_labels, results_values)

mae_aoa =mean_absolute_error(results_labels, aoa_val)
mse_aoa =mean_squared_error(results_labels, aoa_val)

print(f"MAE= Modelo: {mae_model}\t AoA={mae_aoa}")
print(f"MSE= Modelo: {mse_model}\t AoA={mse_aoa}")

MAE= Modelo: 4.4425423596932925	 AoA=17.91555958745627
MSE= Modelo: 61.95675654792057	 AoA=789.3761852881698


In [9]:
model.save_model("./regressor_pd_line.json")

In [12]:
vdataset.pop("training")
vdataset.to_json("../vdataset.json")